In [2]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Importation des données

In [3]:
# Instance_file : str
Instance_file = "InstanceBordeauxV1.xlsx"

# Chargement des donnees
Employees = pd.read_excel(Instance_file, sheet_name= 'Employees')
Tasks = pd.read_excel(Instance_file, sheet_name= 'Tasks')
Employees_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Employees Unavailabilities')
Tasks_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Tasks Unavailabilities')

print(Employees)

  EmployeeName   Latitude  Longitude     Skill  Level WorkingStartTime  \
0     Valentin  45.151218  -0.822093  Oenology      2           8:00am   
1        Ambre  45.151218  -0.822093  Oenology      1           8:00am   

  WorkingEndTime  
0         6:00pm  
1         6:00pm  


## Création des dictionnaires de paramètres

Listes des dictionnaires de paramètres à créer:
- LevelEmployee[i] qui à un employé i lui associe son niveau de compétence
- BeginningEmployee[i] qui à un employé i lui associe son début de journée de travail
- EndEmployee[i] qui à un employé i lui associe sa fin de journée de travail
- LevelTask[j] qui à une tâche j lui associe son niveau de difficulté
- BeginningTask[j] qui à une tâche j lui aussi le début du créneau durant lequel on peut l'effectuer
- EndTask[j] qui à une tâche j lui aussi la fin du créneau durant lequel on peut l'effectuer
- DurationTask[j] qui à une tâche j lui associe sa durée
- TasksUnavailabilities[j]
- EmployeesUnavailabilities[i]
- DistDict[i][j] dictionnaire contenant les distances entre toutes les tâches et les domiciles des employées
- TimeDict[i][j] dictionnaire contenant les temps de trajet associés


In [16]:
LevelEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["Level"][i]
    LevelEmployee[a] = b

#print(LevelEmployee)

BeginningEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingStartTime"][i][:1]
    if Employees["WorkingStartTime"][i][-2:] == 'am':
        BeginningEmployee[a] = b
    if Employees["WorkingStartTime"][i][2:] == "pm":
        BeginningEmployee[a] = b + 12

#print(BeginningEmployee)

EndEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingEndTime"][i][:1]
    if Employees["WorkingEndTime"][i][-2:] == 'am':
        EndEmployee[a] = b
    if Employees["WorkingEndTime"][i][2:] == "pm":
        EndEmployee[a] = b + 12

#print(EndEmployee)

LevelTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["Level"][i]
    LevelTask[a] = b

#print(LevelTask)

BeginningTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = Tasks["TaskId"][i]
    b = Tasks["OpeningTime"][i][:1]
    if Tasks["OpeningTime"][i][-2:] == 'am':
        BeginningTask[a] = b
    if Tasks["OpeningTime"][i][2:] == "pm":
        BeginningTask[a] = b + 12


#print(BeginningTask)


EndTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = Tasks["TaskId"][i] 
    b = Tasks["ClosingTime"][i][:1]
    if Tasks["ClosingTime"][i][-2:] == 'am':
        EndTask[a] = b
    if Tasks["ClosingTime"][i][2:] == "pm":
        EndTask[a] = b + 12

#print(EndTask)

DurationTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["TaskDuration"][i] / 60
    DurationTask[a] = b

#print(DurationTask)


TasksUnavailabilities = dict()

for i in range (len(Tasks_Unavailabilities["TaskId"])):
    a = Tasks_Unavailabilities["TaskId"][i] 
    b = Tasks_Unavailabilities["Start"][i][:1]
    c = Tasks_Unavailabilities["End"][i][:1]
    if Tasks_Unavailabilities["Start"][i][-2:] == 'pm':
        b = b + 12
    if Tasks_Unavailabilities["End"][i][2:] == "pm":
        c = c + 12
    TasksUnavailabilities[a] = [b,c]

#print(TasksUnavailabilities)

EmployeessUnavailabilities = dict()

for i in range (len(Employees_Unavailabilities["EmployeeName"])):
    a = Employees_Unavailabilities["EmployeeName"][i] 
    b = Employees_Unavailabilities["Start"][i][:1]
    c = Employees_Unavailabilities["End"][i][:1]
    if Employees_Unavailabilities["Start"][i][-2:] == 'pm':
        b = b + 12
    if Employees_Unavailabilities["End"][i][2:] == "pm":
        c = c + 12
    TasksUnavailabilities[a] = [b,c]

#print(EmployeessUnavailabilities)


##Création du dictionnaire des distances

DistDict = dict()
R = 6371  #rayon de la Terre

for i in range (len(Tasks["TaskId"])):               ## ajout des distances entre tasks
    for k in range (len(Tasks["TaskId"])):
        if k!=i:
            L_i = Tasks["Longitude"][i]
            l_i = Tasks["Latitude"][i]
            L_k = Tasks["Longitude"][k]
            l_k = Tasks["Latitude"][k]
            A = np.sin((l_i-l_k)*np.pi/(2*180))**2
            B = (np.sin((L_i-L_k)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_k*np.pi/180)
            d = 2*R*np.sqrt(A+B)
            DistDict[(i+1,k)] = d
            DistDict[(k,i+1)] = d

for i in range (len(Tasks["TaskId"])):             #### ajout des disatnces tasks - domiciles des employés
    for k in range (len(Employees["EmployeeName"])):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(i+1, Employees["EmployeeName"][k])] = d 
        DistDict[(Employees["EmployeeName"][k], i+1)] = d       

for i in range (len(Tasks["TaskId"])):
    for k in range (len(Employees_Unavailabilities["EmployeeName"])):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        C = Employees_Unavailabilities["EmployeeName"][k] + "U{}".format(k)   ##############Revoir numerotation des unavailabilities employé pour les retrouver plus vite lors de l'optimisation
        DistDict[(i+1, C)] = d 
        DistDict[(C, i+1)] = d    

#print(DistDict)

##Création dictionnaire des temps de trajet

TimeDict = dict()

for element in DistDict.items():
    t = element[1] / 50
    TimeDict[element[0]] = t

#print(TimeDict)


{(1, 1): 51.08582563186022, (1, 2): 73.07201274303877, (2, 1): 73.07201274303877, (1, 3): 21.986445377956827, (3, 1): 21.986445377956827, (1, 4): 35.44986227176074, (4, 1): 35.44986227176074, (1, 5): 9.770684035139436, (5, 1): 9.770684035139436, (1, 6): 27.9731244554277, (6, 1): 27.9731244554277, (1, 7): 55.462492323540545, (7, 1): 55.462492323540545, (1, 8): 16.80595799156355, (8, 1): 16.80595799156355, (1, 9): 44.39323837943095, (9, 1): 44.39323837943095, (2, 0): 51.08582563186022, (0, 2): 51.08582563186022, (2, 2): 21.986445377956827, (2, 3): 35.44986227176074, (3, 2): 35.44986227176074, (2, 4): 13.727923891840074, (4, 2): 13.727923891840074, (2, 5): 12.229618059745944, (5, 2): 12.229618059745944, (2, 6): 6.226531118680221, (6, 2): 6.226531118680221, (2, 7): 33.550611767536175, (7, 2): 33.550611767536175, (2, 8): 14.596094595567248, (8, 2): 14.596094595567248, (2, 9): 22.900054062964234, (9, 2): 22.900054062964234, (3, 0): 73.07201274303877, (0, 3): 73.07201274303877, (3, 3): 13.727